In [1]:
import os
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import numpy as np
from patchify import (patchify, unpatchify)
from utils import center_crop, rebuild_from_patches

In [2]:
# Display a list with the available models and ask the user to choose which to use
models_path = 'models/f001_w_val_data'
models_list = [name for name in os.listdir(models_path) if os.path.isdir(os.path.join(models_path, name))]
if 'logs' in models_list:
    models_list.remove('logs')

# create a dictionary with numbers as keys and the model names as values
models2dict = dict([(str(a), b) for a, b in enumerate(models_list)])

print('List of Models', '--------------', sep='\n')
for item in models2dict.items():
    print(*item, sep=' --> ')

print('\nChoose a model from the list by typing the number of its key + Enter:')
model_name = models2dict[input()]
print(f'\n{model_name} was selected.')

List of Models
--------------
0 --> unet_input512_filters32_images160_independent_patches_optAdam_lr0.0010_eps150
1 --> unet_input512_filters32_images160_independent_patches_optAdam_lr0.0010_eps500
2 --> unet_input512_filters32_images160_random_patches_optAdam_lr0.0010_eps150
3 --> unet_input512_filters32_images160_random_patches_optAdam_lr0.0010_eps500
4 --> unet_input512_filters32_images589_overlapped_patches_optAdam_lr0.0010_eps150
5 --> unet_input512_filters32_images589_overlapped_patches_optAdam_lr0.0010_eps500
6 --> unet_input512_filters32_images589_random_patches_optAdam_lr0.0010_eps150
7 --> unet_input512_filters32_images589_random_patches_optAdam_lr0.0010_eps500

Choose a model from the list by typing the number of its key + Enter:
5

unet_input512_filters32_images589_overlapped_patches_optAdam_lr0.0010_eps500 was selected.


In [3]:
# Load the model
model = keras.models.load_model(os.path.join(models_path, model_name))

In [4]:
# Retrieve the input layer and extract the size of width of the images (which is equal to the height)
input_layer = model.get_layer(index=0)
input_size = input_layer.input_shape[0][1]

In [5]:
# Load the input and ground truth images 
x_initial_test = plt.imread('test/301-INPUT.jpg')
y_initial_test = plt.imread('test/301-OUTPUT-GT.png')

CROP_SIZE_W = 4000 
CROP_SIZE_H = 4000

x_initial_test = center_crop(x_initial_test, (CROP_SIZE_H, CROP_SIZE_W))
y_initial_test = center_crop(y_initial_test, (CROP_SIZE_H, CROP_SIZE_W))

In [6]:
print(x_initial_test.shape)
print(y_initial_test.shape)

(4000, 4000, 3)
(4000, 4000)


In [ ]:
# Create patches from the input and ground truth images
STEP = 128
input_patches = np.squeeze(patchify(x_initial_test, (input_size, input_size, 3), step=STEP))
ground_truth_patches = np.squeeze(patchify(y_initial_test, (input_size, input_size), step=STEP))
print(input_patches.shape)

In [ ]:
predictions = np.squeeze(model.predict(np.reshape(input_patches, (-1, input_size, input_size, 3))))
predictions = np.reshape(predictions, ground_truth_patches.shape)

In [ ]:
overlapped_images = int(np.power((input_size / STEP), 2))
unified_predictions = rebuild_from_patches(predictions, y_initial_test.shape, STEP, input_size)

In [ ]:
unified_predictions.shape

In [ ]:
plot_side = int(np.sqrt(unified_predictions.shape[-1]))

fig, ax = plt.subplots(plot_side, plot_side, dpi=300)
i = 0
for row in range(plot_side):
    for col in range(plot_side):
        ax[row, col].imshow(unified_predictions[:, :, i])
        ax[row, col].set_xticks([])
        ax[row, col].set_yticks([])
        i += 1
plt.show()

In [ ]:
x = center_crop(x_initial_test, (unified_predictions.shape[:2])) / 255. #NOTE: Later, I must divide by 255 the training image, not here
y = np.expand_dims(center_crop(y_initial_test, (unified_predictions.shape[:2])), axis=-1)

In [ ]:
x_and_y_and_predictions = np.concatenate([x, y, unified_predictions], -1)

name = "prediction_" + model_name
np.save(os.path.join(models_path, name), x_and_y_and_predictions)